In [ ]:
import os
from datetime import datetime
import pickle as pkl
import numpy as np

# PySpark and create Spark context
if not 'sc' in locals():
    import pyspark
    sc = pyspark.SparkContext()

print('done !')

In [ ]:
with open('./var_sparse_tuple_list.pkl', 'rb') as fp:
    spOccur_list = pkl.load(fp)
fp.close()

In [ ]:
print("type of spOccur_list: ", type(spOccur_list))

# data structure of spOccur_list

In [ ]:
# list of triples 
# [
#      ( < sermon-id > , < phrase-id > , < occurance count > )
#      ( < sermon-id > , < phrase-id > , < occurance count > )
#      ( < sermon-id > , < phrase-id > , < occurance count > )
#        ...
#      ( < sermon-id > , < phrase-id > , < occurance count > )
# ]

In [ ]:
with open('./var_phrDict.pkl', 'rb') as fp:
    phrDict = pkl.load(fp) # the phrase dictionary is in sorted order
    dict_pid2phr = {}
    pid = 0
    for phr_ in phrDict:
        dict_pid2phr[pid] = phr_
        pid += 1
fp.close()

In [ ]:
print("type of phrDict: ", type(phrDict))
print("type of dict_pid2phr: ", type(dict_pid2phr))

In [ ]:
phrDict[67600]

In [ ]:
dict_pid2phr.get(67600)

In [ ]:
with open('./var_sermonDict.pkl', 'rb') as fp:
    sermonDict = pkl.load(fp) # the sermon spfn dictionary is in sorted order
    dict_sid2spfn = {}
    sid = 0
    for spfn_ in sermonDict:
        dict_sid2spfn[sid] = spfn_
        sid += 1
fp.close()

In [ ]:
print("type of sermonDict: ", type(sermonDict))

# s2s matrix outlook

In [ ]:
#       0  1  2  3  ...
#     --------------------------
#  0 |  a  b  c  d  ...
#  1 |  e  f  g  h  ...
#  2 |  i  j  k  l  ...
#  . |     .
#  . |     .
#  . |     .

In [ ]:
# -------------------
# sermon-to-phrase
# -------------------

# for i, spc in enumerate(spOccur_list):
#     if spc[2] <30:
#         continue
#     print(f"record {i}    spc: {spc}")
#     if i >= 10000:
#         break

In [ ]:
rdd = sc.parallelize(spOccur_list)

In [ ]:
for spc_ in rdd.take(20):
    print(spc_)

In [ ]:
# rdd :: (sid, pid, cnt)

rdd1 = rdd \
    .map(lambda w: (w[0], (w[1], w[2])))

# rdd1 :: (sid, (pid, cnt))

In [ ]:
rdd_keyBy_sid = rdd1 \
    .groupByKey() \
    .mapValues(list)

In [ ]:
RDD_KEYBY_SID = rdd_keyBy_sid.collect()

In [ ]:
print(f"entry count in RDD_KEYBY_SID: {len(RDD_KEYBY_SID)}")

In [ ]:
# rdd :: (sid, pid, cnt)

rdd2 = rdd \
    .map(lambda w: (w[1], (w[0], w[2])))

# rdd2 :: (pid, (sid, cnt))

In [ ]:
rdd_keyBy_pid = rdd2 \
    .groupByKey() \
    .mapValues(list)

In [ ]:
RDD_KEYBY_PID = rdd_keyBy_pid.collect()

In [ ]:
print(f"entry count in RDD_KEYBY_PID: {len(RDD_KEYBY_PID)}")

In [ ]:
# (s1, p1, a)
# (s1, p2, b)

# (s2, p1, p)
# (s2, p2, q)

# algorithm flow:
# 1. iterate each sermon
#    S_1
#     1.2) check in current sermon S_1 which phrases have occurance
#     1.3) form these phrases in a list
#     1.4) iterate each (other) sermon
#          S_2
#           a) list out phrases occurred in current sermon S_2
#           b) check coocurring phrase

In [ ]:
# -----------------------
# counting criteria 1 :
# -----------------------
#     total phrasal co-occurance from all found phrase

MAX_SID = len(dict_sid2spfn.keys())
MAT_S2S_COOCCUR = np.zeros((MAX_SID, MAX_SID))